<a href="https://colab.research.google.com/github/anderson666r/Riesgo-cardiaco/blob/main/Analisis_patologias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Código fuente

A continuación se explica paso a paso el código implementado, estructurado en tres bloques: carga y preparación de datos, análisis con Spark (DataFrames y RDDs), y simulación del flujo en tiempo real.

Instalación y carga del entorno

In [ ]:
!pip install -q pyspark==3.5.1

Se instala la versión requerida de PySpark en el entorno de Google Colab.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, types as T
from pathlib import Path
import shutil, time
from datetime import datetime
import threading, csv, os

Importación de librerías necesarias para Spark y manejo de archivos en el entorno.

# Carga de datos y creación de la sesión Spark

In [ ]:
spark = SparkSession.builder.appName("HeartFailure-Tarea3").getOrCreate()
from google.colab import files
uploaded = files.upload()

Se crea la sesión de Spark y se carga el archivo local heart.csv en Colab.

In [ ]:
DATA_PATH = "/content/heart.csv"
df = spark.read.csv(DATA_PATH, header=True, inferSchema=True)

Se lee el archivo CSV como un DataFrame de Spark, infiriendo automáticamente los tipos de datos.

# Exploración inicial y limpieza

In [ ]:
df.count()
df.printSchema()
df.show(5, truncate=False)
df.summary().show()

Estas líneas permiten conocer la cantidad de registros, tipos de datos, una vista previa de los datos y estadísticas generales.

In [ ]:
nulls = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
nulls.show(truncate=False)

Se genera una tabla con el conteo de valores nulos por columna.

# Transformaciones clínicas

In [ ]:
# Clasificación por grupo de edad clínica
df = df.withColumn("age_group",
F.when(F.col("Age") < 14, "Niño")
.when((F.col("Age") >= 14) & (F.col("Age") <= 26), "Joven")
.when((F.col("Age") >= 27) & (F.col("Age") <= 59), "Adulto")
.otherwise("Mayor")
)


# Indicadores clínicos de riesgo
df = df.withColumn("bp_risk",
F.when((F.col("RestingBP") < 90) | (F.col("RestingBP") > 120), 1).otherwise(0))


df = df.withColumn("chol_risk",
F.when(F.col("Cholesterol") >= 200, 1).otherwise(0))

Se agregan columnas que clasifican a los pacientes por edad y marcan condiciones de riesgo en presión arterial y colesterol.

# Análisis con DataFrames

In [ ]:
# Conteo por grupo de edad y enfermedad cardíaca
df.groupBy("age_group", "HeartDisease").count().orderBy("age_group", "HeartDisease").show()


# Riesgo por presión y colesterol
df.groupBy("age_group").agg(
F.sum("bp_risk").alias("con_riesgo_presion"),
F.sum("chol_risk").alias("con_riesgo_colesterol")
).orderBy("age_group").show()


# Combinación de factores clínicos
df.groupBy("age_group", "HeartDisease", "FastingBS", "bp_risk", "chol_risk")\
.count().orderBy("age_group").show(truncate=False)

Se realizan distintos agrupamientos para analizar la distribución de los riesgos clínicos por grupo de edad.

# Análisis con RDDs

In [ ]:
rdd = df.select("Age", "HeartDisease", "MaxHR", "bp_risk", "chol_risk").rdd


def en_riesgo(fila):
return (
fila["HeartDisease"] == 1 and
(fila["bp_risk"] == 1 or fila["chol_risk"] == 1) and
(fila["MaxHR"] < 60 or fila["MaxHR"] > 180)
)


rdd_filtrado = rdd.filter(en_riesgo)
print("Pacientes con riesgo múltiple:", rdd_filtrado.count())

Se convierte el DataFrame a RDD para aplicar un filtro más flexible que identifique pacientes con múltiples riesgos simultáneos.

# Simulación de flujo de datos (Kafka)

In [ ]:
import pandas as pd
import os


# División del CSV en lotes
streaming_dir = "/content/streaming_input"
os.makedirs(streaming_dir, exist_ok=True)


batch_size = 100
for i in range((len(df)//batch_size)+1):
batch_df = df.iloc[i*batch_size:(i+1)*batch_size]
batch_df.to_csv(f"{streaming_dir}/batch_{i}.csv", index=False)

Se divide el dataset original en archivos pequeños para simular la llegada progresiva de datos como lo haría Kafka.

# Proceso simulado de Spark Streaming

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pathlib import Path
import time


spark = SparkSession.builder.appName("HeartStreamingSim").getOrCreate()
streaming_dir = Path("/content/streaming_input")
archivos = sorted(streaming_dir.glob("batch_*.csv"))
df_acumulado = None


for archivo in archivos:
df_lote = spark.read.csv(str(archivo), header=True, inferSchema=True)
df_acumulado = df_lote if df_acumulado is None else df_acumulado.union(df_lote)


df_filtrado = df_acumulado.withColumn("chol_risk", F.when(F.col("Cholesterol") >= 200, 1).otherwise(0))
print(f"
🟢 Procesando: {archivo.name}")
print("🔹 Total acumulado:", df_filtrado.count())
print(" - Colesterol alto (≥200):", df_filtrado.filter("chol_risk == 1").count())
print(" - Con enfermedad cardíaca:", df_filtrado.filter("HeartDisease == 1").count())
time.sleep(3)

Este bloque lee cada archivo como si fuera un lote de datos recién llegado, acumula los datos previos y muestra en consola estadísticas acumuladas. El sleep(3) simula un retardo realista entre eventos.